In [69]:
!pip install yfinance

In [70]:
import pandas as pd
import yfinance as yf
from datetime import datetime
import matplotlib.pyplot as plt

In [88]:
symbol = input("Enter stock symbol (e.g., RELIANCE, TCS, INFY): ").upper()
exchange = input("Enter exchange (NS for NSE, BO for BSE): ").upper()
ticker = f"{symbol}.{exchange}"

start_date = input("Enter start date (YYYY-MM-DD): ")
end_date = datetime.today().strftime('%Y-%m-%d')
data = yf.download(ticker, start=start_date, end=end_date)

Enter stock symbol (e.g., RELIANCE, TCS, INFY):  INDHOTEL
Enter exchange (NS for NSE, BO for BSE):  NS
Enter start date (YYYY-MM-DD):  2024-06-01


C:\Users\abali\AppData\Local\Temp\ipykernel_23612\915661245.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed


In [90]:
stock = yf.Ticker(ticker)

# Fetch fundamentals
eps = stock.info.get("trailingEps", None)            # Earnings per Share
pe_ratio = stock.info.get("trailingPE", None)        # Price to Earnings ratio
roe = stock.info.get("returnOnEquity", None)         # Return on Equity (fraction, e.g., 0.18 = 18%)
de_ratio = stock.info.get("debtToEquity", None)      # Debt-to-Equity ratio

print("\nFundamental Metrics:")
print(f"EPS: {eps}")
print(f"P/E Ratio: {pe_ratio}")
print(f"ROE: {roe}")
print(f"Debt-to-Equity: {de_ratio}")


Fundamental Metrics:
EPS: 13.77
P/E Ratio: 55.9114
ROE: 0.18694
Debt-to-Equity: 24.842


In [91]:
# --- Moving Averages ---
data["MA50"] = data["Close"].rolling(50).mean()
data["MA200"] = data["Close"].rolling(200).mean()

# --- RSI Calculation ---
delta = data["Close"].diff()
gain = (delta.where(delta > 0, 0)).rolling(14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(14).mean()

rs = gain / loss
data["RSI"] = 100 - (100 / (1 + rs))

print("\nLatest Technical Indicators:")
print(f"50-day MA: {data['MA50'].iloc[-1]}")
print(f"200-day MA: {data['MA200'].iloc[-1]}")
print(f"RSI: {data['RSI'].iloc[-1]}")


Latest Technical Indicators:
50-day MA: 757.0950158691406
200-day MA: 777.9247827148438
RSI: 57.21154269382341


In [94]:
# 52-week high/low are available in .info

high_52w = stock.info.get("fiftyTwoWeekHigh")
low_52w = stock.info.get("fiftyTwoWeekLow")
current_price = data["Close"].iloc[-1].item()

print("52-week high/low:")
print("52-Week High:", high_52w)
print("52-Week Low:", low_52w)
print("current_price", current_price)

52-week high/low:
52-Week High: 894.9
52-Week Low: 642.05
current_price 769.0499877929688


In [95]:
# Revenue growth (yearly) is available in .info
rev_growth = stock.info.get("revenueGrowth")
if rev_growth:
    rev_growth = rev_growth * 100   # convert to %
    
print("Revenue Growth %:", rev_growth)

Revenue Growth %: 31.7


In [96]:
# Quarterly financials (rows = line items, cols = periods)
fin = stock.quarterly_financials
print(fin)

# Example: Net Income growth (QoQ)
net_income = fin.loc["Net Income"]

if len(net_income) >= 2:
    profit_growth = ((net_income[0] - net_income[1]) / net_income[1]) * 100
    print("Profit Growth % (QoQ):", profit_growth)

                                                      2025-06-30  \
Tax Effect Of Unusual Items                         0.000000e+00   
Tax Rate For Calcs                                  2.737480e-01   
Normalized EBITDA                                   6.371200e+09   
Total Unusual Items                                 0.000000e+00   
Total Unusual Items Excluding Goodwill              0.000000e+00   
Net Income From Continuing Operation Net Minori...  2.963700e+09   
Reconciled Depreciation                             1.427500e+09   
Reconciled Cost Of Revenue                          8.057800e+09   
EBITDA                                              6.371200e+09   
EBIT                                                4.943700e+09   
Net Interest Income                                -5.455000e+08   
Interest Expense                                    5.455000e+08   
Normalized Income                                   2.963700e+09   
Net Income From Continuing And Discontinued Ope.

C:\Users\abali\AppData\Local\Temp\ipykernel_23612\902113202.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  profit_growth = ((net_income[0] - net_income[1]) / net_income[1]) * 100


In [97]:
fin_y = stock.financials
net_income_y = fin_y.loc["Net Income"]

if len(net_income_y) >= 2:
    profit_growth_y = ((net_income_y[0] - net_income_y[1]) / net_income_y[1]) * 100
    print("Profit Growth % (YoY):", profit_growth_y)

Profit Growth % (YoY): 51.5078589752754


C:\Users\abali\AppData\Local\Temp\ipykernel_23612\3278686036.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  profit_growth_y = ((net_income_y[0] - net_income_y[1]) / net_income_y[1]) * 100


In [121]:
def evaluate_stock(eps, pe_ratio, roe, de_ratio, ma50, ma200, rsi, high_52w=None, low_52w=None, current_price=None, rev_growth=None, profit_growth=None, profit_growth_y=None):
    score = 0
    conditions = {}

    # Fundamentals
    if eps and eps > 0:
        score += 1
        conditions["EPS > 0"] = 1
    else:
        conditions["EPS > 0"] = 0

    if pe_ratio and pe_ratio < 20:
        score += 1
        conditions["P/E Ratio < 20"] = 1
    else:
        conditions["P/E Ratio < 20"] = 0

    if roe and roe > 0.15:
        score += 1
        conditions["ROE > 15%"] = 1
    else:
        conditions["ROE > 15%"] = 0

    if de_ratio and de_ratio < 1:
        score += 1
        conditions["D/E Ratio < 1"] = 1
    else:
        conditions["D/E Ratio < 1"] = 0

    # Technicals
    if ma50 and ma200 and ma50 > ma200:
        score += 1
        conditions["MA50 > MA200"] = 1
    else:
        conditions["MA50 > MA200"] = 0

    if rsi and rsi < 70 and rsi > 30:
        score += 1
        conditions["RSI between 30 and 70"] = 1
    else:
        conditions["RSI between 30 and 70"] = 0

    # Price near 52-week high/low (good if closer to high than low)
    if high_52w and low_52w and current_price:
        if current_price > (low_52w + (0.75 * (high_52w - low_52w))):
            score += 1
            conditions["Price near 52-week high"] = 1
        else:
            conditions["Price near 52-week high"] = 0
    else:
        conditions["Price near 52-week high"] = 0

    # Revenue Growth positive
    if rev_growth and rev_growth > 0:
        score += 1
        conditions["Revenue Growth > 0"] = 1
    else:
        conditions["Revenue Growth > 0"] = 0

    # Profit Growth positive QoQ
    if profit_growth and profit_growth > 0:
        score += 1
        conditions["Profit Growth > 0"] = 1
    else:
        conditions["Profit Growth > 0"] = 0

    # Profit Growth positive QoQ
    if profit_growth_y and profit_growth_y > 0:
        score += 1
        conditions["Profit Growth > 0"] = 1
    else:
        conditions["Profit Growth > 0"] = 0
    
    # Final decision
    if score >= 8:
        return "Strong Buy", score, conditions
    elif score >= 6:
        return "Buy / Hold", score, conditions
    else:
        return "Avoid / Weak", score, conditions

In [122]:
result = evaluate_stock(eps, pe_ratio, roe, de_ratio, data["MA50"].iloc[-1], data["MA200"].iloc[-1], data["RSI"].iloc[-1], high_52w, low_52w, current_price,rev_growth, profit_growth, profit_growth_y)
print("Final Recommendation:", result)

Final Recommendation: ('Avoid / Weak', 5, {'EPS > 0': 1, 'P/E Ratio < 20': 0, 'ROE > 15%': 1, 'D/E Ratio < 1': 0, 'MA50 > MA200': 0, 'RSI between 30 and 70': 1, 'Price near 52-week high': 0, 'Revenue Growth > 0': 1, 'Profit Growth > 0': 1})
